In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import tensorflow as tf
import healpy as hp

from deepsphere import HealpyGCNN

from msfm.models.base_model import BaseModel
from msfm.models.delta_model import DeltaLossModel
from msfm.models.nets import small_resnet_partial
from msfm.utils.fiducial_pipeline import get_fiducial_dset
from msfm.utils.input_output import read_yaml
from msfm.utils import survey

import matplotlib.pyplot as plt

### constants

In [3]:
# general
conf_dir = "../../configs/config.yaml"
repo_dir = "../.."
conf = read_yaml(conf_dir)
data_vec_pix, _, _, _, _ = survey.load_pixel_file(conf, repo_dir)

n_side = conf["analysis"]["n_side"]
n_pix = len(data_vec_pix)
n_params = 1 #  Om
pert_indices = np.array([0,7,8])
pert_labels = np.array(conf["analysis"]["fiducial"]["perturbations"]["labels"])[pert_indices]
pert_off_set = [0.01]
print(len(pert_labels), pert_labels)
n_z_bins = 4

# dataset
tfr_pattern = "/Users/arne/data/DESY3/tfrecords/v2/DESy3_fiducial_000.tfrecord"
# tfr_pattern = "/cluster/scratch/athomsen/CosmoGrid/v2/fiducial/DESy3_fiducial_???.tfrecord"
batch_size = 4
examples_shuffle_buffer = 10
n_readers = 1
n_prefetch = 3

23-02-14 10:20:18    survey.py INF   Loaded pixel file 
3 ['fiducial' 'delta_Om_m' 'delta_Om_p']


### set up the fiducial dataset

In [4]:
fiducial_dset = get_fiducial_dset(
    conf,
    repo_dir,
    tfr_pattern,
    pert_labels,
    i_noise=0,
    batch_size=batch_size,
    examples_shuffle_buffer=examples_shuffle_buffer,
    n_readers=n_readers,
    n_prefetch=n_prefetch,
)

23-02-14 10:20:18 fiducial_pip INF   Starting to generate the fiducial training set for i_noise = 0 
23-02-14 10:20:18    survey.py INF   Loaded pixel file 
23-02-14 10:20:18    survey.py INF   Loaded pixel file 
Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-02-14 10:20:18.445582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 10:20:18.445774: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
23-02-14 10:20:18 tfrecords.py WAR   Tracing parse_inverse_fiducial 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the func

### set up the model

In [5]:
network = small_resnet_partial.get_network(n_params)
delta_model = DeltaLossModel(
    network,
    n_side,
    data_vec_pix,
    n_neighbors=8,
    input_shape=(None, n_pix, n_z_bins),
    dir_summary="./summary",
    dir_checkpoint="./checkpoint",
    restore_checkpoint=True
)

23-02-14 10:20:19 delta_model. INF   Initializing DeltaLossModel with a HealpyGCNN model 
Detected a reduction factor of 32.0, the input with nside 512 will be transformed to 16 during a forward pass. Checking for consistency with indices...
indices seem consistent...
Model: "healpy_gcnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 healpy_pseudo_conv (HealpyP  (None, 115968, 32)       544       
 seudoConv)                                                      
                                                                 
 healpy_pseudo_conv_1 (Healp  (None, 28992, 64)        8256      
 yPseudoConv)                                                    
                                                                 
 healpy_pseudo_conv_2 (Healp  (None, 7248, 128)        32896     
 yPseudoConv)                                                    
                                                

### set up the delta loss

In [6]:
delta_model.setup_delta_loss_step(
    n_params=n_params, n_same=batch_size, off_sets=pert_off_set, n_channels=n_z_bins, n_output=n_params
)

### train the model

In [7]:
for data_vec, index in fiducial_dset.take(100):
    delta_model.delta_train_step(data_vec)

# delta_model.save_model()

2023-02-14 10:20:20.821290: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


23-02-14 10:20:22 delta_model. WAR   Tracing delta_train_step 
23-02-14 10:20:22 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 
23-02-14 10:20:24 delta_model. WAR   Tracing delta_train_step 
23-02-14 10:20:24 base_model.p WAR   Performing a base_train_step in python instead of a tf.function 


2023-02-14 10:20:25.049510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
